# Apply SEE  Instances

In [ ]:
# %% import packages
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import ParameterGrid
import inspect

sys.path.append("./source/")

from source.agent_SEE_recycle import SEE_recycle
from source.utils import Linear, Unique_Qualified, OneQuarter_Qualified, AllWorse, AllGood, HalfGood, Experiment_SEE_recycle
from source.env import Environment_Gaussian

Delta = 0.15
Delta_str = str(Delta).replace(".", "p")
filefolder = "./Numeric-Record-Delta_"+ Delta_str + "/"
print(filefolder)
if not os.path.isdir(filefolder):
    os.mkdir(filefolder)

In [ ]:
# %% specify parameter types
instance_type_ = [Linear, Unique_Qualified, OneQuarter_Qualified, AllWorse, AllGood, HalfGood]
K_ = [10, 150, 100, 50, 40, 30, 20, 200]
delta_ = [0.001, 0.0001]

env_ = [Environment_Gaussian]
n_exp = 1000
xi = 0.5

experiment_grid = ParameterGrid(
    {
        "instance_type": instance_type_,
        "agent": [SEE_recycle],
        "env": env_,
        "delta": delta_,
        "K": K_,
    }
)

delta_k_ = lambda x: 1 / 3**x
beta_k_ = lambda x: 2**x / 4
alpha_k_ = lambda x: 5**x
delta_k_str = inspect.getsourcelines(delta_k_)[0][0]
beta_k_str = inspect.getsourcelines(beta_k_)[0][0]
alpha_k_str = inspect.getsourcelines(alpha_k_)[0][0]

In [ ]:
# %% run the experiment
for exp_setting_id, experiment in enumerate(experiment_grid):
    K = experiment["K"]
    delta = experiment["delta"]
    if experiment["instance_type"]==AllWorse:
        inst_fun = experiment["instance_type"]
        rlist = inst_fun(K, Delta=0.25)
    else:
        inst_fun = experiment["instance_type"]
        rlist = inst_fun(K, Delta=Delta)
        
    env_class = experiment["env"]
    agent_class = experiment["agent"]

    numeric_record = pd.DataFrame(
        columns=[
            "K",
            "n_experiment",
            "delta",
            "instance_type",
            "env",
            "agent",
            "success_rate",
            "reward_array",
            "mean_stop_times",
            "std_stop_times",
            "delta_k_",
            "beta_k_",
            "alpha_k_",
            "forced-stop",
            "count_exceed_stop",
            "count_success",
        ]
    )
    stop_benchmark = 1e8
        
    mean_stop_time, std_stop_time, mean_success, _, _, count_exceed_stop, count_success = Experiment_SEE_recycle(
        rlist=rlist,
        delta=delta,
        K=K,
        xi=xi,
        env_class=env_class,
        agent_class=agent_class,
        random_seed_for_start=42,
        n_exp=n_exp,
        disable_tqdm=False,
        delta_k_=delta_k_,
        beta_k_=beta_k_,
        alpha_k_=alpha_k_,
        stop_benchmark=stop_benchmark
    )
    filename = filefolder + agent_class.__name__ + f"beta_{beta_k_(1)}-delta_{delta_k_(1)}.csv"
    numeric_record.loc[0] = np.array(
        [
            K,
            n_exp,
            delta,
            inst_fun.__name__,
            env_class.__name__,
            agent_class.__name__,
            mean_success,
            np.array2string(rlist, threshold=11e3),
            mean_stop_time,
            std_stop_time,
            delta_k_str,
            beta_k_str,
            alpha_k_str,
            stop_benchmark,
            count_exceed_stop,
            count_success
        ],
        dtype=object,
    )
    numeric_record.to_csv(
        filename, mode="a", index=False, header=not os.path.exists(filename)
    )
    print(
        f"{inst_fun.__name__}, {agent_class.__name__}, {env_class.__name__}, {K},{delta}, {exp_setting_id+1}/{len(experiment_grid)} done, stop {mean_stop_time}, success {mean_success}, std {std_stop_time}"
    )